In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from numpy import array
from numpy import split
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os

Use the following to set up the data in the correct format. Explained in data_setup under General

In [5]:
def chunk_and_split_data_to_x_y(df, chunk_size, expected_cols):
    """
    Built to replace both chunk_data and split_data_to_x_y. Does the exact same job but now with a lot less steps.

    Takes in a dataframe whose data you want to use to train a model with. Removes the expected data output from the training dataset.
    Sets up the expected data in the correct format and sets up the training data in the correct format
    Mainly used to organize data for training with a neural network such as keras

    :param df: A dataframe
    :param chunk_size: The size of the 2D matrix that you want to use to train the model. Can handle any non-negative int lower in size than the input dataframe
    :param expected_cols: The expected output columns that are the correct answer for the model. Aka your y
    :return: training data and expected data
    """
    balance_num = df.shape[0] % chunk_size
    expected = df.loc[:, expected_cols]
    expected = expected[chunk_size + balance_num:].values
    expected = expected.reshape(expected.shape[0], 1)
    df = df.drop(columns=expected_cols)
    if balance_num == 0:
        training_data = df.values
        training_data = np.array(np.split(training_data, len(training_data)/chunk_size))
    else:
        df = df.drop(df.index[range(balance_num)])
        training_data = df.values
        training_data = np.array(np.split(training_data, len(training_data)/chunk_size))

    return training_data, expected

df = pd.read_csv("/home/nelson/PycharmProjects/my_notes/Dummy_data_for_examples/flagstaff_solar_data.csv", index_col="date_time", parse_dates=True)
df_val = pd.read_csv("/home/nelson/PycharmProjects/my_notes/Dummy_data_for_examples/flagstaff_val_solar_data.csv", index_col="date_time", parse_dates=True)
# data is already setup from another project


input_chunk_size = 20
expected_data_output_col = "expected_solar"

training_input_data, training_output_data = chunk_and_split_data_to_x_y(df, chunk_size=input_chunk_size,
                                                                        expected_cols=expected_data_output_col)

val_input_data, val_output_data = chunk_and_split_data_to_x_y(df, chunk_size=input_chunk_size,
                                                                        expected_cols=expected_data_output_col)

LAG = 20 # this is the setting for the sliding window of data that the model views
EPOCHS = 10
VERBOSE = 1
BATCH_SIZE = 32

Now to build the model

In [6]:
def build_model(train_x, train_y, val=None, epochs=10, verbose=1, batch_size=32, activation='relu', loss='mse', optimizer='adam',
                save_model_file_path=os.getcwd(), loss_monitor='val_loss'):
    """
    A simple CNN-LSTM that is designed to be used as either an example for how to build a CNN-LSTM or as a ready-made model to test a CNN-LSTM

    :param optimizer: optimizer that the model uses
    :param loss: loss metric
    :param activation: activation function
    :param loss_monitor: method of monitoring the loss
    :param save_model_file_path: path to save the current best performing model
    :param train_x: features that will be used to train the lstm
    :param train_y: expected output
    :param val: validation data
    :param epochs: number of epochs to be used to train the model
    :param verbose: controller for keras if output should be given as the model is trained. Set to true
    :param batch_size: Batch size for the training data.
    :return: the trained model and the model history.
    """
    # define parameters
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    # reshape output into [samples, timesteps, features]
    # train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
    # define model
    model = Sequential()
    model.add(Conv1D(64, 3, activation=activation, input_shape=(n_timesteps, n_features)))
    model.add(Conv1D(64, 3, activation=activation))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(RepeatVector(n_outputs))
    model.add(LSTM(200, activation=activation, return_sequences=True))
    model.add(TimeDistributed(Dense(100, activation=activation)))
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss=loss, optimizer=optimizer)
    filepath = save_model_file_path + "-weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5."
    checkpoint = ModelCheckpoint(filepath, monitor=loss_monitor, verbose=1, save_best_only=True, mode='min')
    es = EarlyStopping(monitor=loss_monitor, mode='min', verbose=1, patience=5)
    callbacks_list = [checkpoint, es]
    # fit network
    history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size,
              verbose=verbose, validation_data=val, callbacks=callbacks_list)
    return model, history

model, history = build_model(training_input_data, training_output_data, val=(val_input_data, val_output_data), epochs=EPOCHS)